In [ ]:
!pip install transformers sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.4 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(task='text2text-generation',model='facebook/m2m100_418M')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('psir.csv' , usecols=['User' , 'Tweets'])

In [ ]:
df.sample(5)

,User,Tweets
2445,HHShkMohd,شهدت اليوم جانباً من فعاليات معرض دبي للطيران....
700,JoeBiden,I've said it many times: Wall Street did not b...
3590,EmmanuelMacron,Russia's war in Ukraine has caused too much su...
2931,lopezobrador_,Avanza la consolidación de un sistema de salud...
3002,lopezobrador_,Nos visitaron en Palacio Nacional integrantes ...


In [ ]:
import re
def remove_url(text):
    """
    remove urls from the tweets
    """
    return re.sub(r"https?://\S+|www\.\S+", "", text)

def remove_html(text):
    """
    remove html tag from the tweets
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)


In [ ]:
df['Tweets'] = df['Tweets'].apply(lambda x : x.lower())
df['Tweets'] = df['Tweets'].apply(lambda x : remove_url(x))
df['Tweets'] = df['Tweets'].apply(lambda x : remove_html(x))

In [ ]:
df[df['User'] == 'HHShkMohd']

,User,Tweets
2004,HHShkMohd,الإخوة والأخوات \n\n17 عاماً مرت على تولينا رئ...
2005,HHShkMohd,ترأست اليوم اجتماع مجلس أمناء مؤسسة مبادرات مح...
2006,HHShkMohd,رحل بجسده .. وبقي بأثره .. ونستذكر كل يوم خيره...
2007,HHShkMohd,استقبلنا مجموعة من العلماء والباحثين … وكرمنا ...
2008,HHShkMohd,وبحسب منظمة التجارة العالمية بلغ تعاملات الإما...
...,...,...
2500,HHShkMohd,"as part of its projects of the 50, the emirate..."
2501,HHShkMohd,مسيرتنا في مجال الفضاء ما تزال في بدايتها .. و...
2502,HHShkMohd,أطلقت دولة الإمارات اليوم ضمن مشاريع الخمسين م...
2503,HHShkMohd,أثناء استقبالي اليوم الرئيس الرواندي بول كاغام...


In [ ]:
df['User'].unique()

array(['narendramodi', 'JoeBiden', 'Pontifex', 'RTErdogan', 'HHShkMohd',
       'KingSalman', 'lopezobrador_', 'EmmanuelMacron'], dtype=object)

In [ ]:
other_lang = ['RTErdogan' ,'HHShkMohd','KingSalman','lopezobrador_','EmmanuelMacron']

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
non_eng_lang = df[df['User'].isin(other_lang)]

In [ ]:
non_eng_lang.shape

(2339, 2)

In [ ]:
non_eng_lang.groupby('User').count()

,Tweets
User,
EmmanuelMacron,501
HHShkMohd,501
KingSalman,335
RTErdogan,501
lopezobrador_,501


In [ ]:
translation_pipeline = pipeline(task='translation',model='Helsinki-NLP/opus-mt-tr-en')

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
translation = []
turks = non_eng_lang[non_eng_lang['User'] == 'RTErdogan']
for i in tqdm(turks['Tweets']):
  translated_sent = translation_pipeline(i)
  translation.append(translated_sent)

  0%|          | 0/501 [00:00<?, ?it/s]

In [ ]:
turks['translated_tweets'] = translation

<ipython-input-47-3c35db45f45a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  turks['translated_tweets'] = translation


In [ ]:
turks.to_csv('turks.csv')

In [ ]:
translation

[[{'translation_text': "You know, you know, you know, you know, you're loyal, you're grateful, you're glorious, you're glorious."}],
 [{'translation_text': 'The national security of the songbird begins outside our borders...'}],
 [{'translation_text': 'And it came to pass, that on the 70th day of the earthquake, we began the building process of over 100,000 dwellings and village houses, and we met our earthquake victims at the banquet, and I gave thanks to every brother of mine, who made us a place at his table.'}],
 [{'translation_text': 'An appointment with our lucky earthquake survivors.'}],
 [{'translation_text': 'We love glory with every member of its land, with every inch of its land, with the courage to give it its glorious title, with the blessing that makes it the center of its charm. Thank you very much.'}],
 [{'translation_text': 'In the last 20 years, we have invested $138 billion in public investments with current figures.'}],
 [{'translation_text': 'We laid the foundation